In [ ]:
#!/usr/bin/env python
# coding: utf-8

In [ ]:
from datetime import datetime as dt
import pandas as pd

In [ ]:
prev = pd.read_csv("prev.csv", encoding='shift-jis', header=0)
curr = pd.read_csv("maps.csv", encoding='shift-jis', header=0)

In [ ]:
n = len(curr) - len(prev)
today = dt.now().isoformat()[:10]

In [ ]:
msg = ["{}、道内で {} 件の新規クラスターが発生しています。".format(today, n)]
for k, v in curr.iterrows():
    s = ""
    if k < len(prev):
        diff = curr.at[k, 'patients'] - prev.at[k, 'patients']
        if diff > 0:
            s = "{}、{}、{}名[+{}名]".format(
                curr.at[k, 'state'], curr.at[k, 'name'],
                curr.at[k, 'patients'], diff)
    else:
        s = "{}、{}、{}名 ※新規".format(
            curr.at[k, 'state'], curr.at[k, 'name'], curr.at[k, 'patients'])
    if s != "":
        msg.append(s)

for m in sorted(msg):
    print(m)

In [ ]:
#prev